In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import xlsxwriter as xlsxwriter
import random
import re
from datetime import datetime
import numpy as np
import pandas as pd



def main():
    # 파일 불러오기
    movie_raw = pd.read_csv("data.csv", dtype={'관객수': np.str_, '스크린수': np.str_, '상영횟수': np.str_})
    movie_raw = movie_raw.loc[:, "영화명": "배우"]
    movie = movie_raw

    # 데이터 추출 후 Nan을 빈값으로 바꿔줌
    movie = movie.fillna('')

    # 엑셀파일 만들기
    outWorkbook = xlsxwriter.Workbook("Distributor" + datetime.today().strftime("%Y%m%d%H%M%S") + ".xlsx")
    worksheet = outWorkbook.add_worksheet("Distributor")

    # 엑셀의 해더를 만든다.
    excel_export_head(worksheet)

    # 엑셀 1행부터 시작하기 위해 선언
    cnt = 1

    # 예외처리 후 프로그램이 돌다 죽을 경우를 대비해서 엑셀파일 생성 여부를 판단하는 구분자.
    excel_yn = False

    # CSV 파일에서 읽은 데이터 프레임을 행별로 뽑는다.
    for index, row in movie.iterrows():

        distribution = row['배급사']
        grade = row['등급']
        genre = row['장르']
        actor = row['배우']
        date = row['개봉일']
        director = row['감독']
        nation = row['대표국적']


        #movie_dic = movie_search_list(row)
        # 영화리스트에서 해당영화를 찾고 데이터 딕셔너리를 리턴한다.
        try:
            movie_dic = movie_search_list(row)

        except Exception as e:
            print("예외처리된 영화명 :", cnt, row['영화명'])
            print('예외가 발생했습니다. 예외처리 내용', e)

            break

        # 받은 데이터 딕셔너리를 엑셀에 출력한다.
        print("크롤링데이터:", cnt, row['영화명'], " -> ", movie_dic)
        print("--------------------------------------------------")
        excel_export(worksheet, cnt, row, movie_dic)

        # 다음행을 가르키기 위해 하나씩 더한다.
        cnt += 1

    outWorkbook.close()

# 엑셀의 헤더
def excel_export_head(worksheet):

    worksheet.write(0, 0, "영화명")
    worksheet.write(0, 1, "개봉일")
    worksheet.write(0, 2, "매출액")
    worksheet.write(0, 3, "누적매출액")
    worksheet.write(0, 4, "관객수")
    worksheet.write(0, 5, "누적관객수")
    worksheet.write(0, 6, "스크린수")
    worksheet.write(0, 7, "상영횟수")
    worksheet.write(0, 8, "대표국적")
    worksheet.write(0, 9, "배급사")
    worksheet.write(0, 10, "등급")
    worksheet.write(0, 11, "장르")
    worksheet.write(0, 12, "감독")
    worksheet.write(0, 13, "배우")
    worksheet.write(0, 14, "별점")
    worksheet.write(0, 15, "참여자수")

# 엑셀 데이터 쓰기
def excel_export(worksheet, cnt, row, movie_dic):
    worksheet.write(cnt, 0, row['영화명'])
    worksheet.write(cnt, 1, row["개봉일"])
    worksheet.write(cnt, 2, row["매출액"])
    worksheet.write(cnt, 3, row["누적매출액"])
    worksheet.write(cnt, 4, row["관객수"])
    worksheet.write(cnt, 5, row["누적관객수"])
    worksheet.write(cnt, 6, row["스크린수"])
    worksheet.write(cnt, 7, row["상영횟수"])
    worksheet.write(cnt, 8, row["대표국적"])
    worksheet.write(cnt, 9, row["배급사"])
    worksheet.write(cnt, 10, row["등급"])
    worksheet.write(cnt, 11, row["장르"])
    worksheet.write(cnt, 12, row["감독"])
    worksheet.write(cnt, 13, row["배우"])
    worksheet.write(cnt, 14, movie_dic["별점"])
    worksheet.write(cnt, 15, movie_dic["참여자수"])




# 네이버 영화리스트 페이지에서 찾고자 하는 영화를 찾는다.
def movie_search_list(row):
    # 딕셔너리 초기화
    movie_dic = {}
    movie_dic['배급사'] = ""
    movie_dic['등급'] = ""
    movie_dic['장르'] = ""
    movie_dic['배우'] = ""
    movie_dic['개봉일'] = ""
    movie_dic['감독'] = ""
    movie_dic['대표국적'] = ""
    movie_dic['별점'] = 0
    movie_dic['참여자수'] = 0

    movie_name = row['영화명']
    director_name = row["감독"]
    nation = row["대표국적"]
    genre = row['장르']

    time_range = random.uniform(0.6,1.1)
    time_range = round(time_range,1)
    time.sleep(time_range)
    print("검색시간 : ",time_range)

    # 네이버 영화 페이지에서 영화명 검색
    url = 'https://movie.naver.com/movie/search/result.naver?section=movie&query=' + str(movie_name)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml', from_encoding='utf-8')

    # 검색되는 영화이름 모두 찾기
    # ul 태그의 클래스 search_list_1 찾음
    items = soup.find("ul", {"class": "search_list_1"})

    # 검색되는 영화가 있을 경우 확인
    if (items != None):
        # ul 태그 아래의 li 태그를 모두 찾음
        li_items = items.find_all('li')

        # 결과값을 담을 리스트 생성
        item_list = []
        for li_item in li_items:

            # 결과값을 담을 딕셔너리 생성
            item_dict = {}
            # 영화제목, URL, 감독 값을 태그에서 찾아 딕셔너리
            # 감독명과 타이틀에서 공백을 제거하고 리스트에 넣는다.
            name_tag = li_item.find('dt')
            name = name_tag.text
            if (name.find('(') > -1):
                name = ''.join(name[:name.find('(') - 1].split())  # 모든 공백제거
                item_dict['타이틀'] = name
            else:
                name = ''.join(name.split())  # 모든 공백제거
                item_dict['타이틀'] = name

            movie_url = name_tag.find('a')['href']
            item_dict['주소'] = movie_url
            dd_items = li_item.find_all("dd", {"class": "etc"})
            # 감독 유무

            if director_name == "" :
                li_text = str(li_item.text)

                if name_match_ratio(item_dict.get('타이틀'), movie_name) == 100 and li_text.find(
                        nation) > -1 and li_text.find(genre) > -1:
                    director = None_director_find(dd_items)
                    item_dict['감독'] = director
                    director_name = director
                else:
                    item_dict['감독'] = None
            else:
                director = None_director_find(dd_items)  # 네이버 크롤링 데이터
                item_dict['감독'] = director

            # 리스트에 딕셔너리를 넣는다
            item_list.append(item_dict)

        #  네이버에서 가져온 영화정보를 CSV 파일에서 가져온 값과 비교를 해서 정말 그 영화인지 찾는다.
        for naver_item in item_list:

            title = naver_item.get('타이틀')
            director = naver_item.get('감독')
            url = str(naver_item.get('주소')).replace('/movie/bi/mi/basic.naver?code=', '')

            director_name = ''.join(str(director_name).split())  # 전처리 데이터의 감독명 공백제거
            movie_name = ''.join(str(movie_name).split())  # 전처리 데이터 영화이름 공백제거

            if (director != None):
                director_ratio = name_match_ratio(director, director_name)  # 크롤링한 감독명과 전처리 데이터의 단어일치율 계산
            else:
                # 감독을 찾고 감동의 일치율 넣어준다.
                director_ratio = 0  # 크롤링 한 대상에 감독명이 없을 경우 0%

            if (title != None):
                movie_name_ratio = name_match_ratio(title, movie_name)  # 크롤링한 감독명과 전처리 데이터의 단어일치율 계산
            else:
                movie_name_ratio = 0  # 크롤링 한 대상에 감독명이 없을 경우 0%

            point = 0
            review_count = 0
            if (director_ratio >= 60 and movie_name_ratio == 100):

                '''
                url = 'https://movie.naver.com/movie/bi/mi/detail.naver?code=' + str(url)
                response = requests.get(url)
                soup = BeautifulSoup(response.content, "lxml", from_encoding="utf-8")
                
                movie_dic['배급사'] = movie_distribution(soup)  # 배급사 찾기함수
                movie_dic['등급'] = movie_grade(soup)  # 등급찾기 함수
                movie_dic['장르'] = movie_genre(soup)  # 장르찾기 함수
                movie_dic['배우'] = movie_actor(soup)  # 배우찾기
                movie_dic['개봉일'] = movie_date(soup)
                movie_dic['감독'] = director_name
                movie_dic['대표국적'] = nation
                '''
                point,review_count = movie_star(soup)

                movie_dic['별점'] = point
                movie_dic['참여자수'] = review_count

                print(' -- 감독명 : ', director, "||", director_name, '-->  일치율: ', director_ratio, "%")
                print(' -- 영화이름 : ', movie_name, "||", title, '-->  일치율: ', movie_name_ratio, "%")
                print(' -- 별점 : ',point, '참여자수: ',review_count)
                print(' -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-')
            else:

                print(' -- 감독명 : ', director, "||", director_name, '--> 일치율: ', director_ratio, "%")
                print(' -- 영화이름 : ', movie_name, "||", title, '--> 일치율: ', movie_name_ratio, "%")
                print(' -- 별점 : ', point, '참여자수: ', review_count)
                print(' -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-')

    return movie_dic

def None_director_find(dd_items):
    for dd_item in dd_items:
        dd_tag = dd_item.text

        if (dd_tag.find('감독') != -1):
            if (dd_tag.find('|') > -1):
                director_tag = dd_tag[:dd_tag.find('|')]
                director = str(director_tag).replace('감독 : ', "")  # 감독 단어제거
                director = ''.join(director.split())  # 모든 공백제거

            else:
                director = dd_tag.replace('감독 : ', "")
                director = ''.join(director.split())
        else:
            director = None

    return director

# 두 단어간 일치 비율을 리턴한다.
def name_match_ratio(word1,word2):

    d1 = list(filter(None,list(word1))) # 공백문자제거
    d1 = list(set(d1).intersection(d1)) # 중복되는 글자 원소 한개만 남김
    d2 = list(filter(None,list(word2))) # 공백문자제거
    d2 = list(set(d2).intersection(d2)) # 중복되는 글자 원소 한개만 남김
    d3 = list(set(d1).intersection(d2)) # 서로 동일한 단어를 리스트화
    if(word1 == word2):
        ratio = 100 #두단어를 비교했는데 정확하게 일치하면 100%
    else:
        if(len(d1) > 0): # 분모가 0 인경우에는 나눌수 없음.
            # 두단어를 비교했는데 같은 글자의 갯수를 찾고
            ratio = int(len(d3) / len(d1) * 100) # 두 리스트 원소갯수를 비교하여 백분율로 변환
        else:
            ratio = 0

    return ratio

# 배급사정보를 html에서 찾는다.
def movie_distribution(soup):
    distribution = ''

    items = soup.find("dl", {"class": "agency_name"})
    # 배급사 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 배급사 정보 체크
            if (i.text == '배급'):
                distribution = str(dd_items[idx].text).replace('\n', '').replace('\t','').replace('\r','')
    return distribution

# 등급을 html에서 찾는다.
def movie_grade(soup):
    grade = ''

    items = soup.find("dl", {"class": "info_spec"})
    # 등급 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 등급 정보 체크
            if (i.text == '등급'):
                grad_line = str(dd_items[idx].text).replace('\n', '').replace('\r', '').replace('\t', '')
                if(grad_line.find('[해외]')> -1):
                    grade = grad_line[:grad_line.find('[해외]')].replace('[국내]','')
                else:
                    grade = grad_line
    return grade

def movie_genre(soup):
    genre = ''

    items = soup.find("dl", {"class": "info_spec"})
    # 장르 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 장르 정보 체크
            if (str(i.text).find('개요') > -1):
                dd_spans = dd_items[idx].find_all('span')
                genre = str(dd_spans[0].text).replace('\n', '').replace('\r', '').replace('\t', '').split(',')[0]
    return genre

def movie_actor(soup):
    actor = ''
    actor_namelist = []
    items = soup.find("ul", {"class": "lst_people"})
    # 배우 정보가 없을 경우 체크
    if (items != None):
        # 모든 배우이름을 찾는다.
        a_items = items.find_all('a', {"class": "k_name"})
        # 배우이름을 담을 리스트를 만든다.

        # 찾은 모든 배우를 찾아 리스트에 담는다.
        for idx, i in enumerate(a_items):
            # 배우 정보 체크
            actor_name = str(i.text)
            actor_namelist.append(actor_name)
            # 4번째부터는 안넣음.
            if(idx == 2):
                break
    if actor_namelist != None:
        actor = ",".join(actor_namelist)

    return actor

def movie_date(soup):
    date = ''

    info_spec = soup.find("dl", {"class": "info_spec"}) # 개봉일이 들어있는 info_spec 를 찾고
    if(info_spec != None):
        span_all_tag = info_spec.find_all('span') # span 중에 하나에 개봉일이 들어 있으니 다 찾고
        if(span_all_tag != None):
            for span_tag in span_all_tag: # 리스트화 되어 있는 span을 반복문에 넣어 하나씩 찾고
                if(span_tag != None):
                    open_day = str(span_tag.text) # span 태그에서 텍스트만 추출하고

                    if(open_day.find('개봉') > -1): # 개봉이라는 글자를 찾고
                        date = re.sub(r"\s+", "", open_day).replace('.', '-').replace('개봉', '').replace('\n', '').replace('\r', '').replace('\t', '') # 모든 공백을 제거하고

                        if(open_day.find('재개봉') > -1):
                            re_open_day = open_day[:open_day.find('재개봉')]
                            date = open_day[open_day.find('재개봉')+4:].replace('\n', '').replace('\r', '').replace('\t', '').replace('.', '-').replace('개봉', '')

    return date
def movie_star(soup):
    point = 0
    review_count = 0

    items = soup.find("dd", {"class": "point"})
    # 장르 정보가 없을 경우 체크
    if (items != None):

        num_count = items.find("em", {"class": "num"})
        cuser_cnt = items.find("em", {"class": "cuser_cnt"})

        if(num_count != None):
            point = str(num_count.text)
            point = float(point)
        if(cuser_cnt != None):
            review_count = str(cuser_cnt.text).replace('(참여 ','').replace('명)','')
            review_count = float(review_count)



    return point, review_count

if __name__ == '__main__':
    main()